In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import tensorflow as tf
# tf.test.gpu_device_name()

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
IMAGE_SIZE = 220

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D

In [ ]:
class ConvNet:
    def __init__(self, numFilters, filterSize, filterConf = 'same'):
        return

    def initModel(self, numFilters, filterSize, filterConf = 'same'):
        model = Sequential()
        input_shape1 = (IMAGE_SIZE, IMAGE_SIZE, 3)
        model.add(Conv2D(numFilters, (filterSize, filterSize), input_shape=input_shape1))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))        
        
        for l in range(4):
            model.add(Conv2D(numFilters, (filterSize, filterSize)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(10, activation='softmax'))
        
        return model

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import cv2
import csv

DATSET_SIZE = 3000

labels = ["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
# Dataset_Path = '/content/drive/MyDrive/inaturalist_12K'
Dataset_Path = os.getcwd() + '/inaturalist_12K'
train_path = os.path.join(Dataset_Path , 'train')
test_path = os.path.join(Dataset_Path , 'val')
def loadImages():
    trainfiles = []
    trainclasses = []
    classcounts = [0 for i in range(10)]
    testfiles = []
    testclasses = []
    for subdir , dirs , files in os.walk(train_path):
        for file in files:
            if file.endswith(".jpg"):
                trainfiles.append(os.path.join(subdir,file))
                c = str(subdir).split(os.sep)[-1]
                for i in range(10):
                    if labels[i] == c:
                        trainclasses.append(i)
                        classcounts[i]+=1
                        break
    for subdir , dirs , files in os.walk(test_path):
        for file in files:
            if file.endswith(".jpg"):
                testfiles.append(os.path.join(subdir,file))
                c = str(subdir).split(os.sep)[-1]
                for i in range(10):
                    if labels[i] == c:
                        testclasses.append(i)
    return trainfiles , trainclasses , testfiles , testclasses , classcounts

def preprocess(data, height, width):
    dim = (width, height)
    resdata = []
    for i in range(len(data[:DATSET_SIZE])):
        try:
            img = cv2.imread(data[i],cv2.IMREAD_UNCHANGED)
            res = cv2.resize(img, dim , interpolation=cv2.INTER_LINEAR)
            #if image is greyscale, convert it to rgb
            if(len(res.shape) != 3):
                res = cv2.cvtColor(res,cv2.COLOR_GRAY2RGB)
            res = res/255.0
            resdata.append(np.asarray(res))
        except Exception as e:
            print(data[i])
            print(str(e))
    return resdata

def dataset(width,height):
    trainfiles , trainclasses , testfiles , testclasses, classcounts = loadImages()
    train_data = preprocess(trainfiles, height,width)
    test_data = preprocess(testfiles, height,width)
    # train_input = []
    # val_input = []
    # train_output = []
    # val_output = []
    # test_output = []
    # test_input = []
    train_output = np.eye(10)[trainclasses]
    test_output = np.eye(10)[testclasses]
    
    L = len(train_data)
    return {
        'Xtrain' : np.array(train_data), 
        'Ytrain' : np.array(train_output),
        # 'Xval' : np.array(val_input),
        # 'Yval' : np.array(val_output),
        'Xtest' :np.array(test_data),
        'Ytest' :np.array(test_output)
    }

def flat(X):
    X_f = []
    for x in X:
        X_f.append(x.flatten())
    return X_f

def savedata(d): 
    with open('train_data.txt','w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(flat(d['Xtrain']))
        
# d = dataset(256,256)
# print(d['Xtrain'])

In [ ]:
network = ConvNet(32, 3)
model = network.initModel(32, 3)
model.summary()

In [ ]:
ds = dataset(IMAGE_SIZE, IMAGE_SIZE)
train_images = ds['Xtrain']
train_labels = ds['Ytrain']

test_images = ds['Xtest']
test_labels = ds['Ytest']

In [ ]:
# plt.imshow(train_images[8030])
# print(train_labels[8030])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels[:3000],
          batch_size=64,
          epochs=20,
          validation_split=0.1,
          verbose=1)

In [ ]:
print(len(train_images))

In [ ]:
# print(len(train_images))
# model.summary()
print(len(train_images))

## Wandb sweep

In [ ]:
import wandb
from wandb.keras import WandbCallback

!wandb login

In [ ]:
def getModel(numFilters, filterSize, filterConf = 'same'):
    model = Sequential()
    input_shape1 = (IMAGE_SIZE, IMAGE_SIZE, 3)
    model.add(Conv2D(numFilters, (filterSize, filterSize), input_shape=input_shape1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))        

    for l in range(4):
        model.add(Conv2D(numFilters, (filterSize, filterSize)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model

In [ ]:
run = wandb.init(project = 'DL-CNN',
                config = {
                    "learning_rate" : 0.005,
                    "epochs" : [20,30],
                    "batch_size" : 64,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN"
                })
config = wandb.config

tf.keras.backend.clear_session()
model = getModel(32, 3)
model.summary()

In [ ]:
# optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
# model.compile(optimizer, config.loss_function, metrics=['acc'])

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_ = model.fit(train_images, train_labels[:10000],
          batch_size=config.batch_size,
          epochs=config.epochs,
          validation_split=0.1,
          verbose=1,
         callbacks = [WandbCallback()])

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test Accuracy: ', round((accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Accuracy': round((accuracy) * 100, 2)})

run.join()
tf.keras.backend.clear_session()

In [ ]:
tf.keras.backend.clear_session()